In [1]:
!pip install "presidio-analyzer[transformers]"
!pip install presidio-anonymizer
!python -m spacy download en_core_web_sm
!python -m spacy download fi_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 31.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fi_core_news_sm')


In [2]:
# this is for spacy only implementation => look into more

#from presidio_analyzer import AnalyzerEngine

# Set up the engine, loads the NLP module (spaCy model by default) and other PII recognizers
#analyzer = AnalyzerEngine()

# Call analyzer to get results
#results = analyzer.analyze(text="My phone number is 212-555-5555",
#                           entities=["PHONE_NUMBER"],
#                           language='en')
#print(results)

In [3]:
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import TransformersNlpEngine
from presidio_anonymizer import AnonymizerEngine

text = "My name is Don and my phone number is 212-555-5555"
text2 = "Minun nimeni on Don ja puhelinnumeroni on 044-210-0000"

# Define which transformers model to use
model_config = [{"lang_code": "en", "model_name": {
    "spacy": "en_core_web_sm",  # use a small spaCy model for lemmas, tokens etc.
    "transformers": "dslim/bert-base-NER"
    }
}]

nlp_engine = TransformersNlpEngine(models=model_config)

# Set up the engine, loads the NLP module (spaCy model by default) 
# and other PII recognizers
analyzer = AnalyzerEngine(nlp_engine=nlp_engine)

# Call analyzer to get results
results = analyzer.analyze(text=text, language='en')
print(results)

# Analyzer results are passed to the AnonymizerEngine for anonymization

anonymizer = AnonymizerEngine()

anonymized_text = anonymizer.anonymize(text=text, analyzer_results=results)

print(anonymized_text)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[type: PERSON, start: 11, end: 14, score: 0.9988730549812317, type: PHONE_NUMBER, start: 38, end: 50, score: 0.75]
text: My name is <PERSON> and my phone number is <PHONE_NUMBER>
items:
[
    {'start': 43, 'end': 57, 'entity_type': 'PHONE_NUMBER', 'text': '<PHONE_NUMBER>', 'operator': 'replace'},
    {'start': 11, 'end': 19, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'}
]

-------
text: Minun nimen<PERSON>n Don ja puhelinnumeroni<PHONE_NUMBER>0000
items:
[
    {'start': 43, 'end': 57, 'entity_type': 'PHONE_NUMBER', 'text': '<PHONE_NUMBER>', 'operator': 'replace'},
    {'start': 11, 'end': 19, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'}
]



In [10]:

#text = "My name is Don and my phone number is 212-555-5555"
text = "Minun nimeni on Don ja puhelinnumeroni on 044-210-0000"

# Define which transformers model to use
model_config = [{"lang_code": "fi", "model_name": {
    "spacy": "fi_core_news_sm",  # use a small spaCy model for lemmas, tokens etc.
    "transformers": "Kansallisarkisto/finbert-ner"},
    "ner_model_configuration":{"model_to_presidio_entity_mapping":
                                {"PER": "PERSON",
                                "PERSON": "PERSON",
                                "LOC": "LOCATION",
                                "LOCATION": "LOCATION",
                                "GPE": "LOCATION",
                                "EVENT":"LOCATION",
                                "ORG": "ORGANIZATION",
                                "DATE": "DATE_TIME",
                                "TIME": "DATE_TIME",
                                "JON": "NRP",
                                "FIBC": "NRP",
                                "NORP": "NRP"}
                              }
}]

nlp_engine = TransformersNlpEngine(models=model_config)

# Set up the engine, loads the NLP module (spaCy model by default) 
# and other PII recognizers
analyzer = AnalyzerEngine(nlp_engine=nlp_engine, allFields=True)

# Call analyzer to get results
results = analyzer.analyze(text=text, language='fi')
print(results)

# Analyzer results are passed to the AnonymizerEngine for anonymization

anonymizer = AnonymizerEngine()

anonymized_text = anonymizer.anonymize(text=text, analyzer_results=results)

print(anonymized_text)


TypeError: AnalyzerEngine.__init__() got an unexpected keyword argument 'allFields'